In [ ]:
'''Check with instructor about instructions,  
all the comparisons asked to explain are the same with 
the exception of scores by school spending''''''

In [1]:
import pandas as pd
import numpy as np

In [2]:
schools = pd.read_csv('Resources/schools_complete.csv')
students = pd.read_csv('Resources/students_complete.csv')
school_data = pd.read_csv('Resources/school_data.csv').drop('Unnamed: 0', axis=1)
student_data = pd.read_csv('Resources/student_data.csv').drop('Unnamed: 0', axis=1)

In [3]:
def suffixes(a):
    return a.strip().replace(
        'Dr. ', '').replace(
        'Mr. ', '').replace(
        'Ms. ', '').replace(
        'Mrs. ', '').replace(
        'Miss ', '').replace(
        'MD ', '').replace(
        'DDS ', '').replace(
        'DVM ', '').replace(
        'PhD ', '')

In [4]:
student_data.student_name = student_data.student_name.apply(suffixes)
student_data.student_name.head()

0       Paul Bradley
1       Victor Smith
2    Kevin Rodriguez
3      Richard Scott
4         Bonnie Ray
Name: student_name, dtype: object

In [5]:
student_data.loc[
    (student_data.school_name == 'Thomas High School') & 
    (student_data.grade == '9th'), 'reading_score'] = np.nan
student_data.loc[
    (student_data.school_name == 'Thomas High School') & 
    (student_data.grade == '9th'), 'math_score'] = np.nan

student_data.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


In [6]:
student_data.isnull().sum()

Student ID         0
student_name       0
gender             0
grade              0
school_name        0
reading_score    461
math_score       461
dtype: int64

In [7]:
school_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
School ID      15 non-null int64
school_name    15 non-null object
type           15 non-null object
size           15 non-null int64
budget         15 non-null int64
dtypes: int64(3), object(2)
memory usage: 728.0+ bytes


In [8]:
student_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 7 columns):
Student ID       39170 non-null int64
student_name     39170 non-null object
gender           39170 non-null object
grade            39170 non-null object
school_name      39170 non-null object
reading_score    38709 non-null float64
math_score       38709 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 2.1+ MB


In [9]:
total = pd.merge(student_data, school_data, on='school_name', how='left')
total.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0,0,District,2917,1910635


In [10]:
total.isnull().sum()

Student ID         0
student_name       0
gender             0
grade              0
school_name        0
reading_score    461
math_score       461
School ID          0
type               0
size               0
budget             0
dtype: int64

In [11]:
student_count = student_data.student_name.count()
school_count = school_data.school_name.count()
total_budget = school_data.budget.sum()

average_reading_score = total.reading_score.mean()
average_math_score = total.math_score.mean()

passing_math = total['math_score'] >= 70
passing_reading = total['reading_score'] >= 70
passing_math = total[passing_math]
passing_reading = total[passing_reading]

passing_math_count = passing_math.student_name.count()
passing_reading_count = passing_reading.student_name.count()
passing_math_reading = total[(total.math_score >= 70) & (total.reading_score >=70)]
overall_passing_math_reading = passing_math_reading.student_name.count()

passing_math_percentage = passing_math_count/float(student_count)*100
passing_reading_percentage = passing_reading_count/float(student_count)*100
overall_passing_percentage = overall_passing_math_reading/student_count*100

district_summary = pd.DataFrame([
    {'total_schools': school_count, 'total_students': student_count,
     'total_budget': total_budget, 'average_math_score': average_math_score,
     'average_reading_score': average_reading_score, 'per_passing_math': passing_math_percentage,
     'per_passing_reading': passing_reading_percentage, 'per_overall_passing': overall_passing_percentage}])

district_summary

,total_schools,total_students,total_budget,average_math_score,average_reading_score,per_passing_math,per_passing_reading,per_overall_passing
0,15,39170,24649428,78.930533,81.855796,73.880521,84.651519,64.092418


In [12]:
per_school_types = school_data.set_index('school_name')['type']
per_school_counts = school_data.set_index('school_name')['size']
per_school_budget = school_data.set_index('school_name')['budget']
per_school_capita = per_school_budget/per_school_counts
student_school_math = student_data.set_index('school_name')['math_score']
per_school_averages = total.groupby('school_name').mean()
per_school_math = total.groupby('school_name').mean()['math_score']
per_school_reading = total.groupby('school_name').mean()['reading_score']
per_school_passing_math = total[total.math_score >= 70]
per_school_passing_reading = total[total.reading_score >= 70]
per_school_passing_math = per_school_passing_math.groupby('school_name').count()['student_name']/per_school_counts*100
per_school_passing_reading = per_school_passing_reading.groupby('school_name').count()['student_name']/per_school_counts*100
per_passing_math_reading = total[(total.math_score >= 70) & (total.reading_score >= 70)]
per_passing_math_reading = per_passing_math_reading.groupby('school_name').count()['student_name']
per_overall_passing_percentage = per_passing_math_reading/per_school_counts*100

school_summary = pd.DataFrame(
    {'school_type': per_school_types, 'total_students':per_school_counts,
     'total_budget': per_school_budget, 'budget_per_student': per_school_capita,
     'avg_math_score': per_school_math, 'avg_reading_score': per_school_reading,
     'per_passing_math': per_school_passing_math, 'per_passing_reading': per_school_passing_reading,
     'per_overall_passing': per_overall_passing_percentage})

school_summary.tail()

,school_type,total_students,total_budget,budget_per_student,avg_math_score,avg_reading_score,per_passing_math,per_passing_reading,per_overall_passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Thomas High School,Charter,1635,1043130,638.0,83.350937,83.896082,66.911315,69.663609,65.076453
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.333333,96.611111,90.333333


In [13]:
top_schools = school_summary.sort_values('per_overall_passing', ascending=False)
bottom_schools = school_summary.sort_values('per_overall_passing')

In [18]:
top_schools

,school_type,total_students,total_budget,budget_per_student,avg_math_score,avg_reading_score,per_passing_math,per_passing_reading,per_overall_passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.333333,96.611111,90.333333
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Thomas High School,Charter,1635,1043130,638.0,83.350937,83.896082,66.911315,69.663609,65.076453
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887


In [14]:
ninth = total[total.grade == '9th']
tenth = total[total.grade == '10th']
eleventh = total[total.grade == '11th']
twelfth = total[total.grade == '12th']

spending_bins = [0,585,615,645,675]
group_names = ['<$584', '$585-629', '$630-644', '$645-675']
school_summary['spending_ranges_per_student'] = pd.cut(per_school_capita, spending_bins, labels=group_names)

spending_math_scores = school_summary.groupby(['spending_ranges_per_student']).mean()['avg_math_score']
spending_reading_scores = school_summary.groupby(['spending_ranges_per_student']).mean()['avg_reading_score']
spending_passing_math = school_summary.groupby(['spending_ranges_per_student']).mean()['per_passing_math']
spending_passing_reading = school_summary.groupby(['spending_ranges_per_student']).mean()['per_passing_reading']
overall_passing_spending = school_summary.groupby(['spending_ranges_per_student']).mean()['per_overall_passing']

spending_summary = pd.DataFrame({
    'avg_math_score': spending_math_scores, 'avg_reading_score': spending_reading_scores,
    'per_passing_math': spending_passing_math, 'per_passing_reading': spending_passing_reading,
    'per_overall_passing': overall_passing_spending})

spending_summary

,avg_math_score,avg_reading_score,per_passing_math,per_passing_reading,per_overall_passing
spending_ranges_per_student,,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,83.599686,83.885211,94.230858,95.900287,90.216324
$630-644,79.067990,81.899294,71.274736,81.499026,61.800133
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [24]:
ninth.groupby('school_name').mean()

,Student ID,reading_score,math_score,School ID,size,budget
school_name,,,,,,
Bailey High School,20344.481481,81.303155,77.083676,7.0,4976.0,3124928.0
Cabrera High School,16969.634470,83.676136,83.094697,6.0,1858.0,1081356.0
Figueroa High School,4397.878505,81.198598,76.403037,1.0,2949.0,1884411.0
Ford High School,36170.595438,80.632653,77.361345,13.0,2739.0,1763916.0
Griffin High School,13031.305623,83.369193,82.044010,4.0,1468.0,917500.0
Hernandez High School,9928.620839,80.866860,77.438495,3.0,4635.0,3022020.0
Holden High School,23068.314961,83.677165,83.787402,8.0,427.0,248087.0
Huang High School,1445.726303,81.290284,77.027251,0.0,2917.0,1910635.0
Johnson High School,32399.975714,81.260714,77.187857,12.0,4761.0,3094650.0


In [16]:
size_bins = [0,1000,2000,5000]
group_names = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

school_summary['school_size'] = pd.cut(school_summary.total_students, size_bins, labels=group_names)
size_math_scores = school_summary.groupby('school_size').mean()['avg_math_score']
size_reading_scores = school_summary.groupby('school_size').mean()['avg_reading_score']
size_passing_math = school_summary.groupby('school_size').mean()['per_passing_math']
size_passing_reading = school_summary.groupby('school_size').mean()['per_passing_reading']
size_overall_passing = school_summary.groupby('school_size').mean()['per_overall_passing']

size_summary = pd.DataFrame({
    'avg_math_score': size_math_scores, 'avg_reading_score': size_reading_scores,
    'per_passing_math': size_passing_math, 'per_passing_reading': size_passing_reading,
    'per_overall_passing': size_overall_passing})

size_summary

,avg_math_score,avg_reading_score,per_passing_math,per_passing_reading,per_overall_passing
school_size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.361201,83.873869,88.327523,91.261628,85.447223
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [17]:
type_math_scores = school_summary.groupby(['school_type']).mean()['avg_math_score']
type_reading_scores = school_summary.groupby(['school_type']).mean()['avg_reading_score']
type_passing_math = school_summary.groupby(['school_type']).mean()['per_passing_math']
type_passing_reading = school_summary.groupby(['school_type']).mean()['per_passing_reading']
type_overall_passing = school_summary.groupby(['school_type']).mean()['per_overall_passing']

type_summary = pd.DataFrame({
    'avg_math_score': type_math_scores, 'avg_reading_score': type_reading_scores,
    'per_passing_math': type_passing_math, 'per_passing_reading': type_passing_reading,
    'per_overall_passing': type_overall_passing})

type_summary

,avg_math_score,avg_reading_score,per_passing_math,per_passing_reading,per_overall_passing
school_type,,,,,
Charter,83.465425,83.902315,90.325723,93.130832,87.198299
District,76.956733,80.966636,66.548453,80.799062,53.672208
